In this particular project, the dataset i have used is the "AmazonProductReviewsData.tsv" which is collected by scraping data from the amazon website. There is a default classification, as to whether the review is a positive or a negetive one in the dataset.
I will be using the Natural Language Toolkit (NLTK) library's VADER (Valence Aware Dictionary and sEntiment Reasoner) sentiment analysis tool, to analyse the sentiment score of each review and based on the compound score generated by VADER, I will manually assign the reviews into positive or negative categories.The results are then tested for accuracy, by comparing it with the default classification.

In the second part of the project, a Machine Learning approach is used, where after preprocessing the reviews, i will be using the TfidfVectorizer from scikit-learn (sklearn) to convert the collection of text data (contained in the preprocessed_reviews variable) into a TF-IDF (Term Frequency-Inverse Document Frequency) matrix representation. This data along with the encoded default calssification is then given as an input to the RandomForest classifier to evaluate the model's performance based on accuracy, precision, and recall.

Importing the necessary Modules and Packages

In [26]:
import os
import numpy as np
import pandas as pd
nltk.download('stopwords')

import nltk
from nltk import corpus, tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer, porter

nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to C:\Users\vibhav
[nltk_data]     bhat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\vibhav
[nltk_data]     bhat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\vibhav
[nltk_data]     bhat\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [27]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [28]:
df = pd.read_csv("AmazonProductReviewsData.tsv", sep='\t')
df.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


In [29]:
df['label'].value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

In [30]:
df.isnull().sum()

label     0
review    0
dtype: int64

I will be using the polarity_scores method of the SentimentIntensityAnalyzer class to compute the sentiment scores for each review in the DataFrame. The scores include a compound score, positive score, negative score, and neutral score.

In [31]:

df['scores']=df['review'].apply(lambda review : sia.polarity_scores(review) )
df.head(10)

,label,review,scores
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co..."
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co..."
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com..."
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com..."
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp..."
5,pos,an absolute masterpiece: I am quite sure any o...,"{'neg': 0.014, 'neu': 0.737, 'pos': 0.249, 'co..."
6,neg,"Buyer beware: This is a self-published book, a...","{'neg': 0.124, 'neu': 0.806, 'pos': 0.069, 'co..."
7,pos,Glorious story: I loved Whisper of the wicked ...,"{'neg': 0.072, 'neu': 0.583, 'pos': 0.346, 'co..."
8,pos,A FIVE STAR BOOK: I just finished reading Whis...,"{'neg': 0.113, 'neu': 0.712, 'pos': 0.174, 'co..."
9,pos,Whispers of the Wicked Saints: This was a easy...,"{'neg': 0.033, 'neu': 0.777, 'pos': 0.19, 'com..."


In [32]:
# Extracting the compound score.
df['compound'] = df['scores'].apply(lambda score_dict : score_dict['compound'])
df.head(10)

,label,review,scores,compound
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781
5,pos,an absolute masterpiece: I am quite sure any o...,"{'neg': 0.014, 'neu': 0.737, 'pos': 0.249, 'co...",0.9900
6,neg,"Buyer beware: This is a self-published book, a...","{'neg': 0.124, 'neu': 0.806, 'pos': 0.069, 'co...",-0.8744
7,pos,Glorious story: I loved Whisper of the wicked ...,"{'neg': 0.072, 'neu': 0.583, 'pos': 0.346, 'co...",0.9900
8,pos,A FIVE STAR BOOK: I just finished reading Whis...,"{'neg': 0.113, 'neu': 0.712, 'pos': 0.174, 'co...",0.8353
9,pos,Whispers of the Wicked Saints: This was a easy...,"{'neg': 0.033, 'neu': 0.777, 'pos': 0.19, 'com...",0.8196


In [33]:
#manually assigning the reviews into positive or negative categories.
df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')
df.head(10)

,label,review,scores,compound,comp_score
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454,pos
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957,pos
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858,pos
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814,pos
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781,pos
5,pos,an absolute masterpiece: I am quite sure any o...,"{'neg': 0.014, 'neu': 0.737, 'pos': 0.249, 'co...",0.9900,pos
6,neg,"Buyer beware: This is a self-published book, a...","{'neg': 0.124, 'neu': 0.806, 'pos': 0.069, 'co...",-0.8744,neg
7,pos,Glorious story: I loved Whisper of the wicked ...,"{'neg': 0.072, 'neu': 0.583, 'pos': 0.346, 'co...",0.9900,pos
8,pos,A FIVE STAR BOOK: I just finished reading Whis...,"{'neg': 0.113, 'neu': 0.712, 'pos': 0.174, 'co...",0.8353,pos
9,pos,Whispers of the Wicked Saints: This was a easy...,"{'neg': 0.033, 'neu': 0.777, 'pos': 0.19, 'com...",0.8196,pos


EVALUATION MATRIX

In [34]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [35]:
print(confusion_matrix(df['label'], df['comp_score']))
print("************"*10)
print(classification_report(df['label'], df['comp_score']))
print("************"*10)
print(accuracy_score(df['label'], df['comp_score']))

[[2629 2468]
 [ 435 4468]]
************************************************************************************************************************
              precision    recall  f1-score   support

         neg       0.86      0.52      0.64      5097
         pos       0.64      0.91      0.75      4903

    accuracy                           0.71     10000
   macro avg       0.75      0.71      0.70     10000
weighted avg       0.75      0.71      0.70     10000

************************************************************************************************************************
0.7097


BUILDING MACHINE LEARNING APPROACH

In [36]:
df = pd.read_csv("AmazonProductReviewsData.tsv", sep='\t')
df.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


PREPROCESSING THE REVIEWS

In [37]:
from tqdm import tqdm

preprocessed_reviews = []

for sentence in tqdm(df['review'].values):
    sentence = re.sub('[^a-zA-Z]',' ',sentence)
    sentence = ' '.join(low.lower() for low in sentence.split() if low.lower() not in stopwords.words('english'))
    preprocessed_reviews.append(sentence.strip())

100%|██████████| 10000/10000 [03:01<00:00, 54.99it/s]


FEATURE EXTRACTION

TF-IDF : Term Frequency - Inverse Documents Frequency

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(preprocessed_reviews).toarray()
pd.DataFrame(x).shape

(10000, 30543)

In [44]:
df['label'] = df['label'].astype('category')
df['label'] = df['label'].cat.codes

In [45]:
df['label'].value_counts()

0    5097
1    4903
Name: label, dtype: int64

In [46]:
# Split the data into training and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, df['label'], test_size=0.25, random_state=1)

In [47]:
# RandomForest Classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

RandomForestClassifier()

In [49]:
y_pred_train = rf.predict(x_train)
y_pred_test = rf.predict(x_test)

In [50]:
print("Training Accuracy :",accuracy_score(y_train, y_pred_train))
print("*************"*10)
print("Test Accuracy :",accuracy_score(y_test, y_pred_test))

Training Accuracy : 1.0
**********************************************************************************************************************************
Test Accuracy : 0.8352


The high performance on training data suggests that the  model has overfit the training data. Therefore i use Cross-Validation to assess the model's performance across multiple validation sets.

In [48]:
# cross validation method becuase the result is found to be overfitting for the 
from sklearn.model_selection import cross_val_score
training_accuracy = cross_val_score(rf, x_train, y_train, cv=5)
training_accuracy.mean()

0.8326666666666667